# Amazon Bedrock AgentCore Runtime で Amazon Bedrock モデルを使用した Pydantic AI エージェントのホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用して既存のエージェントをホストする方法を学びます。Amazon Bedrock モデルと、Azure OpenAI や Gemini などの非 Bedrock モデルを使用した例を提供します。


### チュートリアル詳細


| 情報                | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 会話型                                                                           |
| エージェントタイプ   | シングル                                                                         |
| エージェントフレームワーク | Pydantic AI                                                                   |
| LLM モデル          | Anthropic Claude Sonnet 4                                                        |
| チュートリアル構成要素 | AgentCore Runtime でのエージェントホスティング。Pydantic Agent と Amazon Bedrock Model の使用 |
| チュートリアルの業種 | クロスバーティカル                                                               |
| 例の複雑さ          | 簡単                                                                             |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK と boto3                                      |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した Strands Agent を使用します。

この例では、`get_weather` と `get_date` という2つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主な特徴

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Amazon Bedrock モデルの使用
* Pydantic AI の使用

In [ ]:
!pip install -r requirements.txt

In [ ]:
# AWS リージョンを設定
import os
region="us-east-1" # 使用する AWS リージョンを設定
os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [ ]:
# Jupyter notebook 環境でのみ必要。AgentCore Runtime で実行するエージェントには不要（参照: https://ai.pydantic.dev/troubleshooting/#monitoring-httpx-requests）
import nest_asyncio
nest_asyncio.apply()

## AgentCore Runtime へのデプロイのためのエージェント準備

それでは、エージェントを AgentCore Runtime にデプロイしましょう。そのためには以下が必要です：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* コード内で `app = BedrockAgentCoreApp()` で App を初期化
* `@app.entrypoint` デコレータで呼び出し関数を装飾
* `app.run()` で AgentCore Runtime にエージェントの実行を制御させる

In [ ]:
%%writefile pydantic_bedrock_claude.py

from pydantic_ai.agent import Agent, RunContext

from datetime import datetime
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from pydantic_ai.models.bedrock import BedrockConverseModel

app = BedrockAgentCoreApp()

model = BedrockConverseModel('global.anthropic.claude-haiku-4-5-20251001-v1:0')
dummy_agent = Agent(
    model=model,
    system_prompt="You're a helpful assistant. Use the tools available for you to answer questions."
    
)

@dummy_agent.tool  
def get_current_date(ctx: RunContext[datetime]):
  print("現在の日付を取得中...")
  current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  return current_date


@dummy_agent.tool
def get_weather(ctx: RunContext[str]):
        # シミュレートされた天気データ
  return f"Sunny"

@app.entrypoint
def pydantic_bedrock_claude_main(payload):
  """
   ペイロードを使用してエージェントを呼び出す
  """
  user_input = payload.get("prompt")
  result = dummy_agent.run_sync(user_input)
  print(result.output)
  return result.output


if __name__ == "__main__":
    app.run()

## 舞台裏で何が起こっているか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するための必須の `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラー処理を管理

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name="agentcore_pydantic_bedrockclaude"
response = agentcore_runtime.configure(
    entrypoint="pydantic_bedrock_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### AgentCore Runtime へのエージェント起動

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理して、アプリケーションに組み込むことができます

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### boto3 での AgentCore Runtime の呼び出し

AgentCore Runtime が作成されたので、任意の AWS SDK で呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is today' date?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！